Select data from a file or directory of files. Databricks supports these locations:

Unity Catalog volumes

Workspace files

Cloud object storage

DBFS mounts and DBFS root

Ephemeral storage attached to the driver node of the cluster

Spark SQL and DatabricksSQL
https://docs.databricks.com/en/files/index.html

In [ ]:
#Spark SQL & Databricks SQL

#single file, unity catalog volume (for non tabular data files stored in cloud object storage)
SELECT * FROM csv.`/Volumes/my_catalog/my_schema/my_volume/data.csv`;
LIST '/Volumes/my_catalog/my_schema/my_volume/';


#single file, workspace files (files in a workspace that aren't notebooks. 500MB limit)
SELECT * FROM json.'file:/Workspace/Users/<user-folder>/file.json'

#single file, cloud object store through unity catalog
SELECT * FROM csv.`abfss://container-name@storage-account-name.dfs.core.windows.net/path/file.json`; 
LIST 'abfss://container-name@storage-account-name.dfs.core.windows.net/path';

#single file, from DBFS mount (not securable by Unity Catalog)
SELECT * FROM json.`/mnt/path/to/data.json`;

#from a directory
SELECT * FROM json.`path/to/file.json`

#wildcard match


Apache Spark

In [ ]:
#single file, unity catalog volume (for non tabular data files stored in cloud object storage)
spark.read.format("json").load("/volumes/my_catalog/my_schema/my_volumne/data.json").show()

#single file, workspace files (files in a workspace that aren't notebooks. 500MB limit)
spark.read.format("json").load("file:/Workspace/Users/<user-folder>/data.json").show()

#single file, cloud object store through unity catalog
spark.read.format("json").load("abfss://container-name@storage-account-name.dfs.core.windows.net/path/file.json").show()

#single file, from DBFS mount (not securable by Unity Catalog)
spark.read.format("json").load("/mnt/path/to/data.json").show()

Python/Pandas

In [ ]:
#single file, unity catalog volume (for non tabular data files stored in cloud object storage)
#Pandas
df = pd.read_csv('/Volumes/my_catalog/my_schema/my_volume/data.csv')
#Python
df = spark.read.format("csv").load("/Volumes/catalog_name/schema_name/volume_name/data.csv")

#single file, workspace files (files in a workspace that aren't notebooks. 500MB limit)
df = pd.read_csv('/Workspace/Users/<user-folder>/data.csv')

#cloud object storage.
#Not Supported

#DBFS
df = pd.read_csv('/dbfs/mnt/path/to/data.csv')

##

Create a view, a temporary view, and a CTE as a reference to a file



Create a table from a JDBC connection and from an external CSV file

Deduplicate rows from an existing Delta Lake table

Create a new table from an existing table while removing duplicate rows.

Deduplicate a row based on specific columns.

Parse JSON strings into structs.

Identify the PIVOT clause as a way to convert data from a long format to a wide format

Define a SQL UDF

Identify the location of a function.

readStream

In [ ]:
df_read = (spark.readStream
    .format('cloudFiles')
    .option('cloudFiles.format', 'csv')
    .option('cloudFiles.schemaLocation', {path})
    .option('header', 'true')
    .load('path')
    .schema(schema)
)

In [ ]:
df_read.writeStream(
    .format('delta')
    .option('cloudFiles.checkpointLocation')
    .outputMode('append')
    .query('myQuery_name')
    .trigger(availableNow=True)
    .toTable('my_table_name')
)